In [1]:
!pip install -U -q autogluon > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.13.0 requires aiohttp<4.0.0,>=3.9.2, but you have aiohttp 3.9.1 which is incompatible.
aiobotocore 2.13.0 requires botocore<1.34.107,>=1.34.70, but you have botocore 1.29.165 which is incompatible.
albumentations 1.4.0 requires scikit-image>=0.21.0, but you have scikit-image 0.20.0 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import mutual_info_regression
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [3]:
SEED = 27

In [4]:
train = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/train.csv')
test = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/test.csv')

test_ids = test['id'].values

train.shape, test.shape

((7340, 18), (397, 8))

In [5]:
missing_cols_in_test = [
    'shutdown',
    'mini_shutdown',
    'blackout',
    'mov_change',
    'frankfurt_shutdown',
    'precipitation',
    'snow',
    'user_activity_1',
    'user_activity_2'
]

train = train.drop(missing_cols_in_test, axis=1, errors='ignore')
train.shape, test.shape

((7340, 9), (397, 8))

In [6]:
train['is_train'] = 1
test['is_train'] = 0

combined = pd.concat([train, test], axis=0).reset_index(drop=True)

# Feature Engineering

In [7]:
def add_tfidf_features(df, ngram_range=(1, 4), max_features=300):
    df['holiday_name'] = df['holiday_name'].fillna('')

    vectorizer = TfidfVectorizer(
        tokenizer=lambda x: x,
        preprocessor=lambda x: x,
        token_pattern=None,
        strip_accents='unicode',
        analyzer='word',
        ngram_range=ngram_range,
        sublinear_tf=True,
        max_features=max_features,
    )
    
    vectorized = vectorizer.fit_transform(df['holiday_name'])
    vectorized = vectorized.toarray()
    tfidf_df = pd.DataFrame(vectorized, columns=[f'tfidf_feat_{i}' for i in range(vectorized.shape[1])])
    return pd.concat([df, tfidf_df], axis=1)

In [8]:
def add_count_features(df, ngram_range=(1, 4), max_features=300):
    df['holiday_name'] = df['holiday_name'].fillna('')

    vectorizer = CountVectorizer(
        tokenizer=lambda x: x,
        preprocessor=lambda x: x,
        token_pattern=None,
        strip_accents='unicode',
        analyzer='word',
        ngram_range=ngram_range,
        max_features=max_features,
    )
    
    vectorized = vectorizer.fit_transform(df['holiday_name'])
    vectorized = vectorized.toarray()
    cnt_df = pd.DataFrame(vectorized, columns=[f'cnt_feat_{i}' for i in range(vectorized.shape[1])])
    return pd.concat([df, cnt_df], axis=1)

In [9]:
def add_geo_features(df):
    country_city_mapping = {
        'Prague': 'Czech Republic',
        'Brno': 'Czech Republic',
        'Budapest': 'Hungary',
        'Munich': 'Germany',
        'Frankfurt': 'Germany'
    }

    df['city'] = df['warehouse'].str.split('_').str[0]
    df['country'] = df['city'].map(country_city_mapping)

    return pd.get_dummies(df, columns=['city', 'country'])

In [10]:
def add_time_features(df):    
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)
    df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12)
    df['day'] = df['date'].dt.day
    df['sin_day'] = np.sin(2 * np.pi * df['day'] / 30)
    df['cos_day'] = np.cos(2 * np.pi * df['day'] / 30)
    df['day_of_week'] = df['date'].dt.dayofweek
    df['sin_day_of_week'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['cos_day_of_week'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    df['day_of_year'] = df['date'].dt.dayofyear
    df['sin_day_of_year'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
    df['cos_day_of_year'] = np.cos(2 * np.pi * df['day_of_year'] / 365)
    df['week'] = df['date'].dt.isocalendar().week.astype(int)
    df['quarter'] = df['date'].dt.quarter
    df['season'] = (df['month'] % 12 + 3) // 3
    df['season'] = df['season'].astype(int)
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    df['is_friday'] = df['day_of_week'].isin([4]).astype(int)
    
    df['day_before_holiday'] = df['holiday'].shift().fillna(0)
    df['day_after_holiday'] = df['holiday'].shift(-1).fillna(0)
    df['day_before_school_holiday'] = df['school_holidays'].shift().fillna(0)
    df['day_after_school_holiday'] = df['school_holidays'].shift(-1).fillna(0)
    df['day_before_winter_school_holiday'] = df['winter_school_holidays'].shift().fillna(0)
    df['day_after_winter_school_holiday'] = df['winter_school_holidays'].shift(-1).fillna(0)
    df['holiday_and_shops_closed'] = df['holiday'] * df['shops_closed']

    return df

In [11]:
def convert_cat_features(df):
    return pd.get_dummies(df, columns=['warehouse', 'holiday_name'])

In [12]:
def drop_useless_features(train_df, test_df):
    X_mi = train.drop(['orders', 'id'], axis=1)
    y_mi = train['orders']
    
    mutual_info = mutual_info_regression(X_mi, y_mi, random_state=SEED)
    mutual_info = pd.Series(mutual_info)
    mutual_info.index = X_mi.columns
    mutual_info = pd.DataFrame(mutual_info.sort_values(ascending=False), columns=['Feature_MI'])
    
    useless_cols = []
    for col in mutual_info.index:
        if mutual_info.loc[col, 'Feature_MI'] <= 0:
            useless_cols.append(col)
            
    train_df = train_df.drop(useless_cols, axis=1, errors='ignore')
    test_df = test_df.drop(useless_cols, axis=1, errors='ignore')
    
    return train_df, test_df, useless_cols

In [13]:
def separate_train_test(df):
    train = df[df['is_train'] == 1].drop('is_train', axis=1)
    test = df[df['is_train'] == 0].drop(['is_train', 'orders'], axis=1)

    train = train.set_index('date').sort_index()
    test = test.set_index('date').sort_index()
    
    return train, test

In [14]:
combined = add_tfidf_features(combined)
combined = add_count_features(combined)
combined = add_geo_features(combined)
combined = add_time_features(combined)
combined = convert_cat_features(combined)
train, test = separate_train_test(combined)
train, test, useless_cols = drop_useless_features(train, test)

In [15]:
tfidf_cols = [col for col in train.columns if 'tfidf_feat' in col]
cnt_cols = [col for col in train.columns if 'cnt_feat' in col]

used_tfidf_features = len(tfidf_cols) - len(set(useless_cols) & set(tfidf_cols))
used_cnt_features = len(cnt_cols) - len(set(useless_cols) & set(cnt_cols))

print(f"Used TFIDF features: {used_tfidf_features}/50")
print(f"Used Count features: {used_cnt_features}/50")
print("\nTotal features:", train.shape[1])

Used TFIDF features: 194/50
Used Count features: 172/50

Total features: 423


# Processing Data for AutoGluon

In [16]:
train['item_id'] = train['id'].apply(lambda x: x.split('_')[0] + '_' + x.split('_')[1])
test['item_id'] = test['id'].apply(lambda x: x.split('_')[0] + '_' + x.split('_')[1])

train = train.drop(columns=['id'])
test = test.drop(columns=['id'])

train = train.rename(columns={'orders': 'target'})
test['target'] = 0

train = train.reset_index(drop=False)
test = test.reset_index(drop=False)

In [17]:
train_data = TimeSeriesDataFrame.from_data_frame(train, id_column='item_id', timestamp_column='date')
test_data = TimeSeriesDataFrame.from_data_frame(test, id_column='item_id', timestamp_column='date')

train_data['is_train'] = True
test_data['is_train'] = False

combined = pd.concat([train_data, test_data])

combined = combined.convert_frequency(freq='D')
combined = combined.sort_index().fill_missing_values()

train_data = combined[combined.is_train == True]
test_data = combined[combined.is_train == False]

train_data = train_data.drop(columns=['is_train'])
test_data = test_data.drop(columns=['is_train'])

In [18]:
covariate_features = [col for col in train_data.columns if col not in ['target', 'item_id', 'date']]

# Training

In [19]:
quantile_levels = [0.45, 0.5, 0.55, 0.6, 0.65]

In [20]:
predictor = TimeSeriesPredictor(
    prediction_length=61,
    quantile_levels=quantile_levels,
    target='target',
    eval_metric='MAPE',
    known_covariates_names=covariate_features,
    freq='D'
)

predictor.fit(
    train_data,
    presets='best_quality',
    random_seed=SEED
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240628_080428"
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20240628_080428'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Dec 19 13:14:11 UTC 2023
CPU Count:          4
GPU Count:          1
Memory Avail:       29.79 GB / 31.36 GB (95.0%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Setting presets to: best_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE,
 'freq': 'D',
 'hyperparameters': 'default',
 'known_covariates_names': ['holiday',
                            'shops_closed',
                            'winter_school_holidays',
                            'school_holidays',
                            'tfidf_feat_0',
                            'tfidf_feat_4',
                            'tfidf_feat_5'

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/806M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

	-0.0687       = Validation score (-MAPE)
	26.83   s     = Training runtime
	6.13    s     = Validation (prediction) runtime
Training timeseries model TemporalFusionTransformer. 
	/opt/conda/lib/python3.10/site-packages/torchaudio/lib/libtorchaudio.so: undefined symbol: _ZNK5torch8autograd4Node4nameB5cxx11Ev
Training timeseries model DeepAR. 
	/opt/conda/lib/python3.10/site-packages/torchaudio/lib/libtorchaudio.so: undefined symbol: _ZNK5torch8autograd4Node4nameB5cxx11Ev
Training timeseries model PatchTST. 
	/opt/conda/lib/python3.10/site-packages/torchaudio/lib/libtorchaudio.so: undefined symbol: _ZNK5torch8autograd4Node4nameB5cxx11Ev
Fitting simple weighted ensemble.
	Ensemble weights: {'AutoETS': 0.15, 'Chronos[base]': 0.42, 'DirectTabular': 0.05, 'SeasonalNaive': 0.37}
	-0.0636       = Validation score (-MAPE)
	2.52    s     = Training runtime
	12.43   s     = Validation (prediction) runtime
Training complete. Models trained: ['SeasonalNaive', 'DirectTabular', 'CrostonSBA', 'NPTS',

In [21]:
predictor.leaderboard()

,model,score_val,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-0.063615,12.432360,2.518348,9
1,Chronos[base],-0.068685,6.126931,26.830701,8
2,SeasonalNaive,-0.069710,0.058143,2.368766,1
3,AutoETS,-0.071409,5.074741,16.955899,6
4,DynamicOptimizedTheta,-0.082622,3.076494,24.668491,5
5,AutoARIMA,-0.096447,7.703543,23.112534,7
6,CrostonSBA,-0.108665,0.063192,11.477615,3
7,DirectTabular,-0.139353,1.172545,30.800808,2
8,NPTS,-0.208055,3.273111,2.926304,4


# Making Predictions

In [22]:
predictions = predictor.predict(
    train_data,
    known_covariates=test_data[covariate_features]
)

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [23]:
predictions.to_csv("raw_predictions.csv")
predictions

mean         0.45          0.5         0.55  \
item_id  timestamp                                                        
Brno_1   2024-03-16  9342.413797  9272.350436  9342.413797  9423.389032   
         2024-03-17  8333.104090  8258.188411  8333.104090  8408.019768   
         2024-03-18  8260.802830  8185.498626  8260.802830  8325.195160   
         2024-03-19  8606.669662  8539.878272  8606.669662  8662.549364   
         2024-03-20  8481.839422  8412.920807  8481.839422  8550.758036   
...                          ...          ...          ...          ...   
Prague_3 2024-05-11  5473.851260  5364.884245  5473.851260  5596.844228   
         2024-05-12  5280.575657  5172.097256  5280.575657  5389.053963   
         2024-05-13  5069.795369  4968.049534  5069.795369  5185.567158   
         2024-05-14  4967.951485  4858.133341  4967.951485  5077.769629   
         2024-05-15  4911.380666  4785.699449  4911.380666  5044.074860   

                             0.6         0.65  
item_id  timestamp                             
Brno_1   2024-03-16  9507.546744  9571.485449  
         2024-03-17  8469.202785  8553.938113  
         2024-03-18  8392.873885  8457.527748  
         2024-03-19  8729.028569  8802.264046  
         2024-03-20  8632.735407  8708.205210  
...                          ...          ...  
Prague_3 2024-05-11  5729.259322  5859.671998  
         2024-05-12  5519.426674  5659.495214  
         2024-05-13  5312.328711  5439.444988  
         2024-05-14  5181.439219  5288.589969  
         2024-05-15  5156.599036  5280.406075  

[427 rows x 6 columns]

In [24]:
predictions = predictions.reset_index(drop=False)
predictions['timestamp'] = predictions['timestamp'].astype('str')
predictions['id'] = predictions['item_id'] + '_' + predictions['timestamp']

In [25]:
q = 0
def get_prediction(row):
    return predictions[predictions['id'] == row['id']][str(q)].values[0]

for _q in quantile_levels:
    q = _q
    sub = pd.DataFrame({'id': test_ids, 'orders': 0})
    sub['orders'] = sub.apply(get_prediction, axis=1)
    sub.to_csv(f'submission_{q}.csv', index=False)